In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Import series of helper functions for the notebook (we've created/used these in previous notebooks)
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir

In [ ]:
# Download data from Google Storage (already preformatted)

# !curl -k -o 101_food_classes_10_percent.zip https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip 

# unzip_data("101_food_classes_10_percent.zip")

train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 1550M    0 32884    0     0  36768      0 12:16:47 --:--:-- 12:16:47 36865
  0 1550M    0  502k    0     0   270k      0  1:37:38  0:00:01  1:37:37  271k
  0 1550M    0 10.9M    0     0  3790k      0  0:06:58  0:00:02  0:06:56 3793k
  1 1550M    1 24.9M    0     0  6230k      0  0:04:14  0:00:04  0:04:10 6234k
  2 1550M    2 32.6M    0     0  6680k      0  0:03:57  0:00:05  0:03:52 6682k
  2 1550M    2 45.2M    0     0  7826k      0  0:03:22  0:00:05  0:03:17 9211k
  3 1550M    3 57.1M    0     0  8345k      0  0:03:10  0:00:07  0:03:03 10.9M
  4 1550M    4 72.2M    0     0  9403k      0  0:02:48  0:00:07  0:02:41 12.4M
  5 1550M    5 87.3M    0     0   9.7M      0  0:02:38  0:00:08  0:02:30 12.8M
  6 1550M    6 97.8M    0     0   9.9M      0  0:02

In [4]:
# How many images/classes are there?
walk_through_dir("101_food_classes_10_percent")

There are 2 directories and 0 images in '101_food_classes_10_percent'.
There are 101 directories and 0 images in '101_food_classes_10_percent\test'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\apple_pie'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\baby_back_ribs'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\baklava'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\beef_carpaccio'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\beef_tartare'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\beet_salad'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\beignets'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\bibimbap'.
There are 0 directories and 250 images in '101_food_classes_10_percent\test\bread_pudding'.
There are 0 directories and 250 images in '10

In [5]:
img_size = (224, 224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory??

Signature:
tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    **kwargs,
)
Source:   
@keras_export(
    "keras.utils.image_dataset_from_directory",
    "keras.preprocessing.image_dataset_from_directory",
    v1=[],
)
def image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    **kwargs,
):
    """Generates a `tf.data.Dataset` from image files in a directory.

    If your directory structu